<a href="https://colab.research.google.com/github/rapu34/A1_performance_factory_Aveners/blob/main/AI_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌**2023 연구개발특구 AI SPARK 챌린지 - 사회문제해결형** 
##**부제 : (초미세먼지(PM2.5) 오염도 단기 예측 모델 만들기)**
   
### 팀원: 배재한, 김정현, 김민수

### **1. 구글 드라이브 마운트**

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **2. 필요 라이브러리 및 모듈 호출**



In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### **3-1 데이터 프레임 불러오기**

1.   학습 데이터
2.   기상정보가 포함된 AWS 데이터

3. 결측치 확인

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/dataset (1)/train.csv')
train_df

,연도,일시,측정소,PM2.5
0,0,01-01 00:00,모종동,NaN
1,0,01-01 01:00,모종동,NaN
2,0,01-01 02:00,모종동,NaN
3,0,01-01 03:00,모종동,NaN
4,0,01-01 04:00,모종동,NaN
...,...,...,...,...
596083,3,12-31 19:00,홍성읍,0.060
596084,3,12-31 20:00,홍성읍,0.052
596085,3,12-31 21:00,홍성읍,0.044
596086,3,12-31 22:00,홍성읍,0.052


In [4]:
aws_df = pd.read_csv('/content/drive/MyDrive/dataset (1)/aws.csv')
aws_df

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,계룡,0.154818,0.410833,0.033248,0.0,0.830
1,0,01-01 01:00,계룡,0.142180,0.148611,0.030691,0.0,0.853
2,0,01-01 02:00,계룡,0.173776,0.203889,0.040921,0.0,0.802
3,0,01-01 03:00,계룡,0.169036,0.330556,0.048593,0.0,0.816
4,0,01-01 04:00,계룡,0.169036,0.000000,0.007673,0.0,0.827
...,...,...,...,...,...,...,...,...
1051915,3,12-31 19:00,홍성죽도,0.317536,0.953333,0.230179,0.0,0.630
1051916,3,12-31 20:00,홍성죽도,0.309637,0.984167,0.253197,0.0,0.648
1051917,3,12-31 21:00,홍성죽도,0.308057,0.966667,0.265985,0.0,0.648
1051918,3,12-31 22:00,홍성죽도,0.306477,0.976667,0.235294,0.0,0.642


In [5]:
missing_values = train_df.isnull().sum()

In [6]:
# 초미세먼지 - PM2.5에서 15542의 결측 확인
missing_values

연도           0
일시           0
측정소          0
PM2.5    15542
dtype: int64

### **3-2 데이터 전처리**

1. PM2.5 측정소 위치와 AWS(기상정보) 측정 지점의 위치가 분산되어있으므로 인근 지점으로 매핑 
2. 연도,일시,측정소 컬럼 기준으로 병합

In [7]:
# 30지점 16측정소
aws_df.loc[aws_df['지점'] == "계룡", "지점"] = "노은동"
aws_df.loc[aws_df['지점'] == "세천", "지점"] = "문창동"
aws_df.loc[aws_df['지점'] == "오월드", "지점"] = "정림동"
aws_df.loc[aws_df['지점'] == "장동", "지점"] = "읍내동"
aws_df.loc[aws_df['지점'] == "세종금남", "지점"] = "아름동"
aws_df.loc[aws_df['지점'] == "세종고운", "지점"] = "아름동"
aws_df.loc[aws_df['지점'] == "공주", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "정산", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "유구", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "정안", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "청양", "지점"] = "공주"
aws_df.loc[aws_df['지점'] == "대천항", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "호도", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "춘장대", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "서천", "지점"] = "대천2동"
aws_df.loc[aws_df['지점'] == "논산", "지점"] = "논산"
aws_df.loc[aws_df['지점'] == "양화", "지점"] = "논산"
aws_df.loc[aws_df['지점'] == "홍북", "지점"] = "홍성읍"
aws_df.loc[aws_df['지점'] == "홍성죽도", "지점"] = "홍성읍"
aws_df.loc[aws_df['지점'] == "옹도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "안도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "북격렬비도", "지점"] = "이원면"
aws_df.loc[aws_df['지점'] == "태안", "지점"] = "동문동"
aws_df.loc[aws_df['지점'] == "당진", "지점"] = "동문동"
aws_df.loc[aws_df['지점'] == "대산", "지점"] = "독곶리"
aws_df.loc[aws_df['지점'] == "예산", "지점"] = "예산군"
aws_df.loc[aws_df['지점'] == "아산", "지점"] = "모종동"
aws_df.loc[aws_df['지점'] == "성거", "지점"] = "성성동"
aws_df.loc[aws_df['지점'] == "세종전의", "지점"] = "신방동"
aws_df.loc[aws_df['지점'] == "세종연서", "지점"] = "신흥동"

In [8]:
aws_df.rename(columns = {"지점":"측정소"},inplace=True)
aws_df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827
...,...,...,...,...,...,...,...,...
1051915,3,12-31 19:00,홍성읍,0.317536,0.953333,0.230179,0.0,0.630
1051916,3,12-31 20:00,홍성읍,0.309637,0.984167,0.253197,0.0,0.648
1051917,3,12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648
1051918,3,12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642


In [9]:
merged_df = pd.merge(aws_df, train_df, on=['연도','일시','측정소'])
merged_df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...,...
1051915,3,12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3,12-31 22:00,홍성읍,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3,12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3,12-31 23:00,홍성읍,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [10]:
# 잘 병합되었는지 확인
x = merged_df.loc[merged_df["측정소"]=="모종동","PM2.5"]
x

771408      NaN
771409      NaN
771410      NaN
771411      NaN
771412      NaN
          ...  
806467    0.036
806468    0.048
806469    0.044
806470    0.048
806471    0.044
Name: PM2.5, Length: 35064, dtype: float64

In [11]:
merged_df.측정소.unique()

array(['노은동', '이원면', '성성동', '동문동', '대천2동', '독곶리', '논산', '공주', '아름동',
       '예산군', '모종동', '신방동', '읍내동', '문창동', '정림동', '신흥동', '홍성읍'],
      dtype=object)

In [12]:
aws_df.측정소.unique()

array(['노은동', '이원면', '성성동', '동문동', '대천2동', '독곶리', '논산', '공주', '아름동',
       '예산군', '모종동', '신방동', '읍내동', '문창동', '정림동', '신흥동', '홍성읍'],
      dtype=object)

In [13]:
# 베이스라인 학습 데이터셋 구성
merged_df.to_csv('final_df.csv')

### **4. 시계열 데이터 가공**

1. [일시] 컬럼을 연-월-일 시:분으로 재정의 
  - Datetime 모듈 사용
  - pd.to_Datime 모듈 사용
2. label Encoder를 사용하여 범주형 변수 수치로 변환

In [14]:
# 더미 데이터프레임으로 카피하여 사용
df = merged_df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051920 entries, 0 to 1051919
Data columns (total 9 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   연도       1051920 non-null  int64  
 1   일시       1051920 non-null  object 
 2   측정소      1051920 non-null  object 
 3   기온(°C)   1032972 non-null  float64
 4   풍향(deg)  1032972 non-null  float64
 5   풍속(m/s)  1032972 non-null  float64
 6   강수량(mm)  1032972 non-null  float64
 7   습도(%)    1032972 non-null  float64
 8   PM2.5    1023621 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 80.3+ MB


In [15]:
# 결측치 확인
df.isna().sum()

연도             0
일시             0
측정소            0
기온(°C)     18948
풍향(deg)    18948
풍속(m/s)    18948
강수량(mm)    18948
습도(%)      18948
PM2.5      28299
dtype: int64

In [20]:
df[df['PM2.5'].isnull()&df['기온(°C)'].isnull()] #결측치가 중구난방임

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
12354,1,05-30 18:00,노은동,NaN,NaN,NaN,NaN,NaN,NaN
23494,2,09-06 22:00,노은동,NaN,NaN,NaN,NaN,NaN,NaN
23763,2,09-18 03:00,노은동,NaN,NaN,NaN,NaN,NaN,NaN
23764,2,09-18 04:00,노은동,NaN,NaN,NaN,NaN,NaN,NaN
31711,3,08-14 07:00,노은동,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1003800,1,04-04 12:00,홍성읍,NaN,NaN,NaN,NaN,NaN,NaN
1003802,1,04-04 13:00,홍성읍,NaN,NaN,NaN,NaN,NaN,NaN
1003804,1,04-04 14:00,홍성읍,NaN,NaN,NaN,NaN,NaN,NaN
1003806,1,04-04 15:00,홍성읍,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df[df['기온(°C)'].isnull()&df['풍향(deg)'].isnull()]

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
135,0,01-06 15:00,노은동,NaN,NaN,NaN,NaN,NaN,0.156
136,0,01-06 16:00,노은동,NaN,NaN,NaN,NaN,NaN,0.116
137,0,01-06 17:00,노은동,NaN,NaN,NaN,NaN,NaN,0.104
138,0,01-06 18:00,노은동,NaN,NaN,NaN,NaN,NaN,0.124
139,0,01-06 19:00,노은동,NaN,NaN,NaN,NaN,NaN,0.132
...,...,...,...,...,...,...,...,...,...
1051811,3,12-29 17:00,홍성읍,NaN,NaN,NaN,NaN,NaN,0.312
1051824,3,12-30 00:00,홍성읍,NaN,NaN,NaN,NaN,NaN,0.356
1051826,3,12-30 01:00,홍성읍,NaN,NaN,NaN,NaN,NaN,0.340
1051828,3,12-30 02:00,홍성읍,NaN,NaN,NaN,NaN,NaN,0.392


#결측치 처리
1. 전 주에 평균치로 떄려박자. -> 애초에 미세먼지 오는 주가 날라갔을 수 있기 때문에 무의미 하다고 판단.

2. 걍 날리자 -> 이걸로감

In [17]:
df[df['측정소']=='노은동']

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,노은동,0.265403,0.101944,0.025575,0.0,0.596,0.032
35060,3,12-31 20:00,노은동,0.252765,0.338056,0.015345,0.0,0.594,0.048
35061,3,12-31 21:00,노은동,0.257504,0.970556,0.040921,0.0,0.575,0.020
35062,3,12-31 22:00,노은동,0.262243,0.908056,0.030691,0.0,0.611,0.056


In [18]:
# 임시로 결측치 날려
df = df.dropna()

In [19]:
# 데이터 확인
df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0,01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0,01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0,01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0,01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...,...
1051915,3,12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3,12-31 22:00,홍성읍,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3,12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3,12-31 23:00,홍성읍,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [20]:
# 2020년에 2월29일이 존재함으로 임의로 년도 정의 및 매핑
df.loc[df['연도'] == 0, '일시'] = '2017-' + df.loc[df['연도'] == 0, '일시']
df.loc[df['연도'] == 1, '일시'] = '2018-' + df.loc[df['연도'] == 1, '일시']
df.loc[df['연도'] == 2, '일시'] = '2019-' + df.loc[df['연도'] == 2, '일시']
df.loc[df['연도'] == 3, '일시'] = '2020-' + df.loc[df['연도'] == 3, '일시']

In [21]:
df

,연도,일시,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,2017-01-01 00:00,노은동,0.154818,0.410833,0.033248,0.0,0.830,0.052
1,0,2017-01-01 01:00,노은동,0.142180,0.148611,0.030691,0.0,0.853,0.072
2,0,2017-01-01 02:00,노은동,0.173776,0.203889,0.040921,0.0,0.802,0.068
3,0,2017-01-01 03:00,노은동,0.169036,0.330556,0.048593,0.0,0.816,0.064
4,0,2017-01-01 04:00,노은동,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...,...
1051915,3,2020-12-31 21:00,홍성읍,0.308057,0.966667,0.265985,0.0,0.648,0.044
1051916,3,2020-12-31 22:00,홍성읍,0.262243,0.866944,0.043478,0.0,0.725,0.052
1051917,3,2020-12-31 22:00,홍성읍,0.306477,0.976667,0.235294,0.0,0.642,0.052
1051918,3,2020-12-31 23:00,홍성읍,0.257504,0.000000,0.000000,0.0,0.710,0.060


In [22]:
# Datetime 형식으로 변환
df['일시'] = df['일시'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M'))

In [23]:
# 변환되었는지 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005259 entries, 0 to 1051919
Data columns (total 9 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   연도       1005259 non-null  int64         
 1   일시       1005259 non-null  datetime64[ns]
 2   측정소      1005259 non-null  object        
 3   기온(°C)   1005259 non-null  float64       
 4   풍향(deg)  1005259 non-null  float64       
 5   풍속(m/s)  1005259 non-null  float64       
 6   강수량(mm)  1005259 non-null  float64       
 7   습도(%)    1005259 non-null  float64       
 8   PM2.5    1005259 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 76.7+ MB


In [22]:
# 시계열 학습을 위해 ["일시"] 컬럼을 인덱스로 설정한다.
df = df.set_index(["일시"])

In [27]:
# labelencoder를 사용하여 범주형 변수를 수치형으로 변환한다.
le = LabelEncoder()
df['측정소'] = le.fit_transform(df['측정소'])
df

,연도,측정소,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
일시,,,,,,,,
2017-01-01 00:00:00,0,1,0.154818,0.410833,0.033248,0.0,0.830,0.052
2017-01-01 01:00:00,0,1,0.142180,0.148611,0.030691,0.0,0.853,0.072
2017-01-01 02:00:00,0,1,0.173776,0.203889,0.040921,0.0,0.802,0.068
2017-01-01 03:00:00,0,1,0.169036,0.330556,0.048593,0.0,0.816,0.064
2017-01-01 04:00:00,0,1,0.169036,0.000000,0.007673,0.0,0.827,0.064
...,...,...,...,...,...,...,...,...
2020-12-31 21:00:00,3,16,0.308057,0.966667,0.265985,0.0,0.648,0.044
2020-12-31 22:00:00,3,16,0.262243,0.866944,0.043478,0.0,0.725,0.052
2020-12-31 22:00:00,3,16,0.306477,0.976667,0.235294,0.0,0.642,0.052


In [28]:
df['PM2.5'].isnull().sum()

0

## 5. **AI 모델링**

- 타깃 변수 지정과 x, y 설정
- train_test_split 사용으로 학습과 테스트데이터 분류

In [28]:
target = "PM2.5"
y = df[target]
x = df.drop(columns = target)

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

###LSTM 사용 및 MAE 평가


In [29]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

y = df['PM2.5']
x = df.drop(columns='PM2.5')

train_data = TimeseriesGenerator(x, y, length=24, batch_size=32)

# 2. LSTM 모델 훈련
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(32, input_shape=(24, x.shape[1])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(train_data, epochs=2)

Epoch 1/2


<ipython-input-29-5d4530e20447>:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, epochs=2)


31414/31414 [==============================] - 195s 6ms/step - loss: 0.0045
Epoch 2/2
31414/31414 [==============================] - 182s 6ms/step - loss: 0.0041


In [ ]:
# 3. 모델 성능 평가
from sklearn.metrics import mean_absolute_error

test_data = TimeseriesGenerator(x_test, y_test, length=24, batch_size=1)

y_pred = model.predict_generator(test_data)
mae = mean_absolute_error(y_test[24:], y_pred[:-1])
print('MAE: ', mae)

<ipython-input-30-e9c5ee9e1347>:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(test_data)
